In [174]:
import os
import pandas as pd
dataset_path = '../data/nutrition5k_reconstructed/'

dataset_path = '../../data/nutrition5k_reconstructed/'

image_path = os.path.join(dataset_path, 'images')
label_path = os.path.join(dataset_path, 'labels/labels.csv')
train_path = os.path.join(dataset_path, 'metadata/train_ids.csv')
test_path = os.path.join(dataset_path, 'metadata/test_ids.csv')

# Training Files
with open(train_path, 'r') as f:
    train_files = f.read().splitlines()
    print('Number of training files:', len(train_files))
    
# Testing Files
with open(test_path, 'r') as f:
    test_files = f.read().splitlines()
    print('Number of testing files:', len(test_files))

# Labels
labels = pd.read_csv(label_path)
print('Number of labels:', len(labels))
print('\n------------------------------------\n')

# Displaying the first 5 rows of the labels
print(labels.head())

# Record original labels
original_labels = labels.copy()

# select the first 5 columns
labels = labels.iloc[:, :6]
labels.head()


Number of training files: 2755
Number of testing files: 507
Number of labels: 3262

------------------------------------

                id    calories   mass        fat       carb    protein  \
0  dish_1562699612   98.391266   74.0   2.671786  15.661942   2.793359   
1  dish_1558722322   37.720001   92.0   0.184000   9.200000   0.828000   
2  dish_1567007848  100.300003  243.0   0.580000  24.836000   1.655000   
3  dish_1563898084  419.493378  384.0  22.837477  30.001255  24.971926   
4  dish_1561406762   17.205421   35.0   0.414091   3.452406   0.310967   

   brown rice     quinoa  olive oil     carrot  ...  pepperoni  \
0        22.0  50.485435   1.514563   0.000000  ...        0.0   
1         0.0   0.000000   0.000000  92.000000  ...        0.0   
2         0.0   0.000000   0.000000   0.000000  ...        0.0   
3         0.0   0.000000   0.000000   0.000000  ...        0.0   
4         0.0   0.000000   0.345040  34.504005  ...        0.0   

   orange with peel  mozzarella chee

,id,calories,mass,fat,carb,protein
0,dish_1562699612,98.391266,74.0,2.671786,15.661942,2.793359
1,dish_1558722322,37.720001,92.0,0.184000,9.200000,0.828000
2,dish_1567007848,100.300003,243.0,0.580000,24.836000,1.655000
3,dish_1563898084,419.493378,384.0,22.837477,30.001255,24.971926
4,dish_1561406762,17.205421,35.0,0.414091,3.452406,0.310967


In [175]:
# stats about the labels
stats = labels.describe()
stats

,calories,mass,fat,carb,protein
count,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000
mean,255.012738,214.980074,12.727633,19.386111,18.004492
std,219.637570,161.497428,13.526409,21.605225,20.044535
min,0.000000,1.000000,0.000000,0.000000,0.000000
25%,80.114996,92.000000,1.845926,6.171500,2.128136
50%,209.110062,177.000000,8.845456,15.360000,11.488166
75%,375.122963,305.000000,19.032520,27.532451,28.034391
max,3943.325195,3051.000000,106.343002,844.568604,147.491821


In [176]:
import numpy as np
import pandas as pd

def normalization_method(labels, log_min_max=True):
    if log_min_max:
        # Log Transformation with a small constant to avoid log(0)
        labels.iloc[:, 1:6] = np.log1p(labels.iloc[:, 1:6])

        print('Log Transformation:')
        print(labels.head(3))
        print(labels.describe())

        # Normalizing outputs in range [0-1] using max-min normalization
        min_values = labels.iloc[:, 1:6].min()
        max_values = labels.iloc[:, 1:6].max()
        labels.iloc[:, 1:6] = labels.iloc[:, 1:6].sub(min_values).div(max_values - min_values)

        # Saving the minimum and maximum values for inverse transformation
        with open('min_max_values.csv', 'w') as f:
            f.write("category,min,max\n")
            for i, (min_val, max_val) in enumerate(zip(min_values, max_values)):
                f.write(f"{labels.columns[i + 1]},{min_val},{max_val}\n")

        print('\n------------------------------------\n')
        print('Normalized labels:')
        print(labels.head(3))

        # Adding full image name column...
        labels['img_indx'] = labels["id"] + ".jpeg"

        # Add original labels to the dataframe
        labels['original_calories'] = original_labels['calories']
        labels['original_mass'] = original_labels['mass']
        labels['original_fat'] = original_labels['fat']
        labels['original_carb'] = original_labels['carb']
        labels['original_protein'] = original_labels['protein']

    else:
        # Log Transformation with a small constant to avoid log(0)
        labels.iloc[:, 1:6] = np.log1p(labels.iloc[:, 1:6])
        # Adding full image name column...
        labels['img_indx'] = labels["id"] + ".jpeg"

        # Add original labels to the dataframe
        labels['original_calories'] = original_labels['calories']
        labels['original_mass'] = original_labels['mass']
        labels['original_fat'] = original_labels['fat']
        labels['original_carb'] = original_labels['carb']
        labels['original_protein'] = original_labels['protein']
        print('Log Transformation:')
        print(labels.head(3))
        print(labels.describe())

    
    return labels

labels = normalization_method(labels, log_min_max=True)


Log Transformation:
                id  calories      mass       fat      carb   protein
0  dish_1562699612  4.599064  4.317488  1.300678  2.813127  1.333252
1  dish_1558722322  3.656356  4.532599  0.168899  2.322388  0.603222
2  dish_1567007848  4.618086  5.497168  0.457425  3.251769  0.976445
          calories         mass          fat         carb      protein
count  3262.000000  3262.000000  3262.000000  3262.000000  3262.000000
mean      5.086636     5.080756     2.045539     2.633352     2.309239
std       1.112333     0.836975     1.192373     0.948255     1.231602
min       0.000000     0.693147     0.000000     0.000000     0.000000
25%       4.395868     4.532599     1.045888     1.970112     1.140437
50%       5.347631     5.181784     2.287010     2.794839     2.524780
75%       5.929916     5.723585     2.997357     3.351042     3.368481
max       8.280033     8.023552     4.676029     6.740009     5.000530

------------------------------------

Normalized labels:
       

In [177]:
# stats about the labels
stats = labels.describe()
stats

,calories,mass,fat,carb,protein,original_calories,original_mass,original_fat,original_carb,original_protein
count,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000
mean,0.614326,0.598549,0.437452,0.390705,0.461799,255.012738,214.980074,12.727633,19.386111,18.004492
std,0.134339,0.114179,0.254997,0.140690,0.246294,219.637570,161.497428,13.526409,21.605225,20.044535
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.530900,0.523771,0.223670,0.292301,0.228063,80.114996,92.000000,1.845926,6.171500,2.128136
50%,0.645847,0.612331,0.489092,0.414664,0.504903,209.110062,177.000000,8.845456,15.360000,11.488166
75%,0.716171,0.686243,0.641005,0.497187,0.673625,375.122963,305.000000,19.032520,27.532451,28.034391
max,1.000000,1.000000,1.000000,1.000000,1.000000,3943.325195,3051.000000,106.343002,844.568604,147.491821


In [178]:
# Splitting the data into training, validation and testing
from sklearn.model_selection import train_test_split

# Create Training Dataframe
train_labels = labels[labels['id'].isin(train_files)]
print('Number of training labels:', len(train_labels))

# Create Testing Dataframe
test_labels = labels[labels['id'].isin(test_files)]

# Splitting the test data into validation and testing
test_labels, val_labels = train_test_split(test_files, test_size=0.4, random_state=42)
print('Number of validation labels:', len(val_labels))
print('Number of testing labels:', len(test_labels))
# convert the validation and testing labels to dataframes
val_labels = labels[labels['id'].isin(val_labels)]
test_labels = labels[labels['id'].isin(test_labels)]

# Saving the training, validation and testing labels
base_dir = './data'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

train_labels.to_csv(os.path.join(base_dir, 'train_labels_lmm.csv'), index=False)
val_labels.to_csv(os.path.join(base_dir, 'val_labels_lmm.csv'), index=False)
test_labels.to_csv(os.path.join(base_dir, 'test_labels_lmm.csv'), index=False)

Number of training labels: 2755
Number of validation labels: 203
Number of testing labels: 304


In [179]:
# Deal with ingredients - only select id and columns 6 to end
ingr_labels = original_labels.iloc[:, 6:]
ingr_labels['id'] = original_labels['id']
# move id to the first column
cols = ingr_labels.columns.tolist()
cols = cols[-1:] + cols[:-1]
ingr_labels = ingr_labels[cols]
ingr_labels.head()

,id,brown rice,quinoa,olive oil,carrot,watermelon,raspberries,berries,cantaloupe,pineapple,...,pepperoni,orange with peel,mozzarella cheese,baby carrots,banana with peel,wheat bread,chilaquiles,pasta salad,balsamic vinegar,toast
0,dish_1562699612,22.0,50.485435,1.514563,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dish_1558722322,0.0,0.000000,0.000000,92.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dish_1567007848,0.0,0.000000,0.000000,0.000000,91.0,19.0,45.0,42.0,46.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dish_1563898084,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,119.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dish_1561406762,0.0,0.000000,0.345040,34.504005,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
# stats about the labels
stats = ingr_labels.describe()
stats

,brown rice,quinoa,olive oil,carrot,watermelon,raspberries,berries,cantaloupe,pineapple,bacon,...,pepperoni,orange with peel,mozzarella cheese,baby carrots,banana with peel,wheat bread,chilaquiles,pasta salad,balsamic vinegar,toast
count,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,...,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000
mean,1.496598,2.082282,2.254085,4.698533,3.101165,0.304558,5.290006,3.138259,3.867534,1.541079,...,0.047823,0.020540,0.007971,0.017474,0.038013,0.025138,0.161251,0.069896,0.002759,0.025751
std,7.874031,15.661577,4.183736,17.638863,17.726781,4.680894,25.656135,17.254722,18.556931,7.430017,...,1.931083,1.173094,0.455231,0.576022,2.171099,1.435727,5.010299,2.822353,0.157580,1.470745
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,3.179672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,127.430267,212.992554,50.000000,147.874298,203.000000,156.000000,423.000000,300.000000,188.000000,92.000000,...,78.000000,67.000000,26.000000,19.000000,124.000000,82.000000,190.000000,114.000000,9.000000,84.000000


In [181]:
# for every dish, if the ingredient is present, set it to 1, else 0
# except for the first column which is the id
ingr_indicater_df = ingr_labels.copy()
ingr_indicater_df.iloc[:, 1:] = np.where(ingr_indicater_df.iloc[:, 1:] > 0, 1, 0)
# Adding full image name column...
ingr_indicater_df['img_indx'] = ingr_indicater_df["id"] + ".jpeg"
ingr_indicater_df.head()

,id,brown rice,quinoa,olive oil,carrot,watermelon,raspberries,berries,cantaloupe,pineapple,...,orange with peel,mozzarella cheese,baby carrots,banana with peel,wheat bread,chilaquiles,pasta salad,balsamic vinegar,toast,img_indx
0,dish_1562699612,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1562699612.jpeg
1,dish_1558722322,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1558722322.jpeg
2,dish_1567007848,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1567007848.jpeg
3,dish_1563898084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1563898084.jpeg
4,dish_1561406762,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1561406762.jpeg


In [182]:
# apply the normalization method to the ingredients
import numpy as np

def normalization_method(labels, log_min_max=True):
    if log_min_max:
        # Log Transformation with a small constant to avoid log(0)
        labels.iloc[:, 1:] = np.log1p(labels.iloc[:, 1:])

        # Normalizing outputs in range [0-1] using max-min normalization
        min_values = labels.iloc[:, 1:].min()
        max_values = labels.iloc[:, 1:].max()
        labels.iloc[:, 1:] = labels.iloc[:, 1:].sub(min_values).div(max_values - min_values)

        # Saving the minimum and maximum values for inverse transformation
        with open('min_max_values_ingr.csv', 'w') as f:
            f.write("category,min,max\n")
            for i, (min_val, max_val) in enumerate(zip(min_values, max_values)):
                f.write(f"{labels.columns[i + 1]},{min_val},{max_val}\n")

        print('\n------------------------------------\n')
        print('Normalized labels:')
        print(labels.head(3))

        # Adding full image name column...
        labels['img_indx'] = labels["id"] + ".jpeg"
    else:
        # Log Transformation with a small constant to avoid log(0)
        labels.iloc[:, 1:] = np.log1p(labels.iloc[:, 1:])
        # Adding full image name column...
        labels['img_indx'] = labels["id"] + ".jpeg"
        
        print('Log Transformation:')

    
    return labels

ingr_labels = normalization_method(ingr_labels, log_min_max=False)
ingr_labels.head(3)


Log Transformation:


,id,brown rice,quinoa,olive oil,carrot,watermelon,raspberries,berries,cantaloupe,pineapple,...,orange with peel,mozzarella cheese,baby carrots,banana with peel,wheat bread,chilaquiles,pasta salad,balsamic vinegar,toast,img_indx
0,dish_1562699612,3.135494,3.941299,0.922099,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1562699612.jpeg
1,dish_1558722322,0.000000,0.000000,0.000000,4.532599,0.000000,0.000000,0.000000,0.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1558722322.jpeg
2,dish_1567007848,0.000000,0.000000,0.000000,0.000000,4.521789,2.995732,3.828641,3.7612,3.850148,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,dish_1567007848.jpeg


In [183]:
# stats about the labels
stats = ingr_labels.describe()
stats

,brown rice,quinoa,olive oil,carrot,watermelon,raspberries,berries,cantaloupe,pineapple,bacon,...,pepperoni,orange with peel,mozzarella cheese,baby carrots,banana with peel,wheat bread,chilaquiles,pasta salad,balsamic vinegar,toast
count,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,...,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000,3262.000000
mean,0.193351,0.138661,0.698663,0.381860,0.150738,0.040732,0.244495,0.172176,0.219188,0.183848,...,0.002679,0.001294,0.001010,0.002755,0.001480,0.001355,0.005805,0.002909,0.000706,0.001362
std,0.742536,0.704209,0.892794,1.092938,0.809593,0.317758,1.020899,0.845488,0.933804,0.751411,...,0.108177,0.073879,0.057706,0.090821,0.084538,0.077369,0.167151,0.117473,0.040316,0.077786
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.430232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4.855386,5.365941,3.931826,5.003102,5.318120,5.056246,6.049733,5.707110,5.241747,4.532599,...,4.369448,4.219508,3.295837,2.995732,4.828314,4.418841,5.252273,4.744932,2.302585,4.442651


In [184]:

# Splitting the data into training, validation and testing
from sklearn.model_selection import train_test_split

# Create Training Dataframe
train_labels = ingr_labels[ingr_labels['id'].isin(train_files)]
train_id_labels = ingr_indicater_df[ingr_indicater_df['id'].isin(train_files)]
print('Number of training labels:', len(train_labels))

# Create Testing Dataframe
test_labels = ingr_labels[ingr_labels['id'].isin(test_files)]
test_id_labels = ingr_indicater_df[ingr_indicater_df['id'].isin(test_files)]

# Splitting the test data into validation and testing
test_labels, val_labels = train_test_split(test_files, test_size=0.4, random_state=42)
print('Number of validation labels:', len(val_labels))
print('Number of testing labels:', len(test_labels))
# convert the validation and testing labels to dataframes
val_labels = ingr_labels[ingr_labels['id'].isin(val_labels)]
test_labels = ingr_labels[ingr_labels['id'].isin(test_labels)]

val_id_labels = ingr_indicater_df[ingr_indicater_df['id'].isin(val_labels['id'])]
test_id_labels = ingr_indicater_df[ingr_indicater_df['id'].isin(test_labels['id'])]

# Saving the training, validation and testing labels
base_dir = './data'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

train_labels.to_csv(os.path.join(base_dir, 'train_labels_ingr_log.csv'), index=False)
val_labels.to_csv(os.path.join(base_dir, 'val_labels_ingr_log.csv'), index=False)
test_labels.to_csv(os.path.join(base_dir, 'test_labels_ingr_log.csv'), index=False)

train_id_labels.to_csv(os.path.join(base_dir, 'train_id_labels.csv'), index=False)
val_id_labels.to_csv(os.path.join(base_dir, 'val_id_labels.csv'), index=False)
test_id_labels.to_csv(os.path.join(base_dir, 'test_id_labels.csv'), index=False)

Number of training labels: 2755
Number of validation labels: 203
Number of testing labels: 304
